In [ ]:
!pip install tensorflow

In [10]:
import cv2, os, json, argparse

import numpy as np
import tensorflow as tf

In [11]:
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

In [12]:
parser = argparse.ArgumentParser()
parser.add_argument('--image')
args, unknown = parser.parse_known_args()

# Constants

In [13]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male','Female']

# Directory paths to models
model_dir_path = os.path.join(os.getcwd(), "models")
gender_model_dirpath = os.path.join(model_dir_path, "gender")
age_model_dirpath = os.path.join(model_dir_path, "age")
face_model_dirpath = os.path.join(model_dir_path, "opencv_face")
celeb_model_dirpath = os.path.join(model_dir_path, "celeb_similarity")
emotion_model_dirpath = os.path.join(model_dir_path, "emotion")

# File path to models
face_model_path = os.path.join(face_model_dirpath, "opencv_face_detector_uint8.pb")
face_model_text_path = os.path.join(face_model_dirpath, "opencv_face_detector.pbtxt")

age_model_path = os.path.join(age_model_dirpath, "age_net.caffemodel")
age_model_text_path = os.path.join(age_model_dirpath, "age_deploy.prototxt")

gender_model_path = os.path.join(gender_model_dirpath, "gender_net.caffemodel")
gender_model_text_path = os.path.join(gender_model_dirpath, "gender_deploy.prototxt")

celeb_model_path = os.path.join(celeb_model_dirpath, "mobilenet.h5")
celeb_model_classes_path = os.path.join(celeb_model_dirpath, "celeb_classes.json")

# Models

In [14]:
faceNet=cv2.dnn.readNet(face_model_path, face_model_text_path)
ageNet=cv2.dnn.readNet(age_model_path, age_model_text_path)
genderNet=cv2.dnn.readNet(gender_model_path, gender_model_text_path)

tf_emotion_model = tf.keras.models.load_model(emotion_model_path) 
tf_celeb_model = tf.keras.models.load_model(celeb_model_path)

In [15]:
celebs_option = []
with open(celeb_model_classes_path, 'r') as infile:
    celebs_option = json.loads(json.load(infile))
    celebs_option = list(map(lambda x: x.split("_")[-1], celebs_option))

# Helper Functions

In [16]:
def predict_celebrity(face):
    resized_img = cv2.resize(face, (160,160), interpolation = cv2.INTER_AREA)
    resized_img = np.reshape(resized_img,(-1,160,160,3))/255.0
    result = np.argmax(tf_celeb_model.predict(resized_img), axis=-1)
    return celebs_option[result[-1]]

def predict_gender(faceBlob):
    genderNet.setInput(faceBlob)
    genderPreds=genderNet.forward()
    gender=genderList[genderPreds[0].argmax()]
    return gender

def predict_age(faceBlob):
    ageNet.setInput(faceBlob)
    agePreds=ageNet.forward()
    age=ageList[agePreds[0].argmax()]
    return age
    
def moodNamePrintFromLabel(n):
    if n == 0: result = 'Angry '
    elif n == 1: result = 'Disgust '
    elif n == 2: result = 'Fear'
    elif n == 3: result = 'Happy'
    elif n == 4: result = 'Sad'
    elif n == 5: result = 'Surprise'
    elif n == 6: result = 'Neutral'
    return result

def predict_emotion(face):
    resized_img = cv2.resize(face, (224,224), interpolation = cv2.INTER_AREA)
    resized_img = np.reshape(resized_img,(-1,224,224,3))/255.0
    result = np.argmax(tf_emotion_model.predict(resized_img), axis=-1)
    return moodNamePrintFromLabel(result)

# Main

In [17]:
video = cv2.VideoCapture(args.image if args.image else 0)
padding = 20

continue_program = True

while continue_program:
    hasFrame, frame = video.read()
    if not hasFrame:
        continue
    continue_program = False
    resultImg, faceBoxes = highlightFace(faceNet, frame)
    if not faceBoxes:
        continue_program = True
        print("No face detected")

    for faceBox in faceBoxes:
        face = frame[max(0, faceBox[1]-padding):
                     min(faceBox[3]+padding, frame.shape[0]-1), max(0, faceBox[0]-padding):min(faceBox[2]+padding, frame.shape[1]-1)]

        blob = cv2.dnn.blobFromImage(
            face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        gender = predict_gender(blob)
        age = predict_age(blob)
        predicted_actor = predict_celebrity(face)
        emotion = predict_emotion(face)

        text = f'{gender}, {age}, {predicted_actor}, {emotion}'
        x0 = faceBox[0]
        y0 = faceBox[1]
        cv2.putText(resultImg, text, (x0, y0),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("window", resultImg)
        key = cv2.waitKey(10)
        continue_program = True
        if key == 27:
            continue_program = False
            break

cv2.destroyAllWindows()
video.release()

1/1 [==============================] - 0s 76ms/step
No face detected
No face detected
1/1 [==============================] - 0s 75ms/step
No face detected
1/1 [==============================] - 0s 96ms/step
